### **Pull data from fastf1 API**

In [7]:
path = '/content/'
data_path = '/content/drive/MyDrive/F1 Fantasy/Data'

In [4]:
!pip install fastf1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.8/128.8 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 29.9 MB/s eta 0:00:00


In [8]:
import os
os.chdir(path)
import pandas as pd
import logging
import json
import pickle
import requests
import numpy as np
import fastf1
import matplotlib.pyplot as plt
import pickle
from fastf1.core import DataNotLoadedError

fastf1.Cache.enable_cache(data_path)

In [9]:
def get_lap_data(session, schedule, evnt, ses, EventDate):
    '''get lap data from a session and
       retun as a dataframe
    '''
    if len(session.laps)==0:
        return []
    lap = session.laps.to_dict()
    lap = pd.DataFrame.from_dict(lap)
    lap['RoundNumber'] = schedule['RoundNumber'][schedule['EventName']==evnt].values[0]
    lap['EventName'] = evnt
    lap['country'] = schedule['Country'][schedule['EventName']==evnt].values[0]
    lap['session'] = ses
    lap['EventDate'] = schedule[EventDate][schedule['EventName']==evnt].values[0]
    return lap

def get_weather_data(session, schedule, evnt, ses,EventDate):
    '''get weather data from a session and
       retun as a dataframe
    '''
    if len(session.weather_data)==0:
        return []
    weather = session.weather_data.to_dict()
    weather = pd.DataFrame.from_dict(weather)
    weather['RoundNumber'] = schedule['RoundNumber'][schedule['EventName']==evnt].values[0]
    weather['EventName'] = evnt
    weather['country'] = schedule['Country'][schedule['EventName']==evnt].values[0]
    weather['session'] = ses
    weather['EventDate'] = schedule[EventDate][schedule['EventName']==evnt].values[0]
    return weather

def get_car_data(session, schedule, evnt, ses,EventDate):
    '''get car_data from a session and
       retun as a dataframe
    '''
    if len(session.car_data)==0:
        return []
    session.weather_data
    for ii in session.car_data:
        car_data = session.car_data[ii].to_dict()
        car_data = pd.DataFrame.from_dict(car_data)
        car_data['driver'] = ii
    car_data['RoundNumber'] = schedule['RoundNumber'][schedule['EventName']==evnt].values[0]
    car_data['EventName'] = evnt
    car_data['country'] = schedule['Country'][schedule['EventName']==evnt].values[0]
    car_data['session'] = ses
    car_data['EventDate'] = schedule[EventDate][schedule['EventName']==evnt].values[0]
    return car_data

def get_position_data(session, schedule, evnt, ses, EventDate):
    '''get position_data from a session and
       retun as a dataframe
    '''
    if len(session.pos_data)==0:
        return []
    for ii in session.pos_data:
        position = session.pos_data[ii].to_dict()
        position = pd.DataFrame.from_dict(position)
        position['driver'] = ii
    position['RoundNumber'] = schedule['RoundNumber'][schedule['EventName']==evnt].values[0]
    position['EventName'] = evnt
    position['country'] = schedule['Country'][schedule['EventName']==evnt].values[0]
    position['session'] = ses
    position['EventDate'] = schedule[EventDate][schedule['EventName']==evnt].values[0]
    return position

def get_results(session, schedule, evnt, ses, EventDate):
    '''get results data from a session and
       retun as a dataframe
    '''
    if len(session.results)==0:
        return []
    result = session.results.to_dict()
    result = pd.DataFrame.from_dict(result).reset_index()
    result['RoundNumber'] = schedule['RoundNumber'][schedule['EventName']==evnt].values[0]
    result['EventName'] = evnt
    result['country'] = schedule['Country'][schedule['EventName']==evnt].values[0]
    result['session'] = ses
    result['EventDate'] = schedule[EventDate][schedule['EventName']==evnt].values[0]
    return result

In [10]:
fastf1.get_event_schedule(2022)

,RoundNumber,Country,Location,OfficialEventName,EventDate,EventName,EventFormat,Session1,Session1Date,Session1DateUtc,...,Session3,Session3Date,Session3DateUtc,Session4,Session4Date,Session4DateUtc,Session5,Session5Date,Session5DateUtc,F1ApiSupport
0,0,Spain,Spain,FORMULA 1 PRE-SEASON TRACK SESSION 2022,2022-02-25,Pre-Season Track Session,testing,Practice 1,2022-02-23 09:00:00+01:00,2022-02-23 08:00:00,...,Practice 3,2022-02-25 09:00:00+01:00,2022-02-25 08:00:00,None,NaT,NaT,None,NaT,NaT,False
1,0,Bahrain,Bahrain,FORMULA 1 ARAMCO PRE-SEASON TESTING 2022,2022-03-12,Pre-Season Test,testing,Practice 1,2022-03-10 10:00:00+03:00,2022-03-10 07:00:00,...,Practice 3,2022-03-12 10:00:00+03:00,2022-03-12 07:00:00,None,NaT,NaT,None,NaT,NaT,True
2,1,Bahrain,Sakhir,FORMULA 1 GULF AIR BAHRAIN GRAND PRIX 2022,2022-03-20,Bahrain Grand Prix,conventional,Practice 1,2022-03-18 15:00:00+03:00,2022-03-18 12:00:00,...,Practice 3,2022-03-19 15:00:00+03:00,2022-03-19 12:00:00,Qualifying,2022-03-19 18:00:00+03:00,2022-03-19 15:00:00,Race,2022-03-20 18:00:00+03:00,2022-03-20 15:00:00,True
3,2,Saudi Arabia,Jeddah,FORMULA 1 STC SAUDI ARABIAN GRAND PRIX 2022,2022-03-27,Saudi Arabian Grand Prix,conventional,Practice 1,2022-03-25 17:00:00+03:00,2022-03-25 14:00:00,...,Practice 3,2022-03-26 17:00:00+03:00,2022-03-26 14:00:00,Qualifying,2022-03-26 20:00:00+03:00,2022-03-26 17:00:00,Race,2022-03-27 20:00:00+03:00,2022-03-27 17:00:00,True
4,3,Australia,Melbourne,FORMULA 1 HEINEKEN AUSTRALIAN GRAND PRIX 2022,2022-04-10,Australian Grand Prix,conventional,Practice 1,2022-04-08 13:00:00+10:00,2022-04-08 03:00:00,...,Practice 3,2022-04-09 13:00:00+10:00,2022-04-09 03:00:00,Qualifying,2022-04-09 16:00:00+10:00,2022-04-09 06:00:00,Race,2022-04-10 15:00:00+10:00,2022-04-10 05:00:00,True
5,4,Italy,Imola,FORMULA 1 ROLEX GRAN PREMIO DEL MADE IN ITALY ...,2022-04-24,Emilia Romagna Grand Prix,sprint,Practice 1,2022-04-22 13:30:00+02:00,2022-04-22 11:30:00,...,Practice 2,2022-04-23 12:30:00+02:00,2022-04-23 10:30:00,Sprint,2022-04-23 16:30:00+02:00,2022-04-23 14:30:00,Race,2022-04-24 15:00:00+02:00,2022-04-24 13:00:00,True
6,5,United States,Miami,FORMULA 1 CRYPTO.COM MIAMI GRAND PRIX 2022,2022-05-08,Miami Grand Prix,conventional,Practice 1,2022-05-06 14:30:00-04:00,2022-05-06 18:30:00,...,Practice 3,2022-05-07 13:00:00-04:00,2022-05-07 17:00:00,Qualifying,2022-05-07 16:00:00-04:00,2022-05-07 20:00:00,Race,2022-05-08 15:30:00-04:00,2022-05-08 19:30:00,True
7,6,Spain,Barcelona,FORMULA 1 PIRELLI GRAN PREMIO DE ESPAÑA 2022,2022-05-22,Spanish Grand Prix,conventional,Practice 1,2022-05-20 14:00:00+02:00,2022-05-20 12:00:00,...,Practice 3,2022-05-21 13:00:00+02:00,2022-05-21 11:00:00,Qualifying,2022-05-21 16:00:00+02:00,2022-05-21 14:00:00,Race,2022-05-22 15:00:00+02:00,2022-05-22 13:00:00,True
8,7,Monaco,Monaco,FORMULA 1 GRAND PRIX DE MONACO 2022,2022-05-29,Monaco Grand Prix,conventional,Practice 1,2022-05-27 14:00:00+02:00,2022-05-27 12:00:00,...,Practice 3,2022-05-28 13:00:00+02:00,2022-05-28 11:00:00,Qualifying,2022-05-28 16:00:00+02:00,2022-05-28 14:00:00,Race,2022-05-29 15:00:00+02:00,2022-05-29 13:00:00,True
9,8,Azerbaijan,Baku,FORMULA 1 AZERBAIJAN GRAND PRIX 2022,2022-06-12,Azerbaijan Grand Prix,conventional,Practice 1,2022-06-10 15:00:00+04:00,2022-06-10 11:00:00,...,Practice 3,2022-06-11 15:00:00+04:00,2022-06-11 11:00:00,Qualifying,2022-06-11 18:00:00+04:00,2022-06-11 14:00:00,Race,2022-06-12 15:00:00+04:00,2022-06-12 11:00:00,True


In [14]:
sched = fastf1.get_event_schedule(2021)
sched.loc[sched['EventName']=='Bahrain Grand Prix','EventFormat'].values[0]

logger      WARNING 	Failed to load schedule from FastF1 backend!
DEBUG:fastf1.fastf1.events:Traceback for failure in FastF1 schedule
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/fastf1/logger.py", line 151, in __wrapped
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/fastf1/events.py", line 556, in _get_schedule_ff1
    response = Cache.requests_get(
  File "/usr/local/lib/python3.10/dist-packages/fastf1/req.py", line 294, in requests_get
    return cls._requests_session_cached.get(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/requests_cache/session.py", line 126, in get
    return self.request('GET', url, params=params, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/requests_cache/session.py", line 182, in request
    return super().request(method, url, *args, headers=headers, **kwargs)  # type: ignore
  File "/usr/local/lib/python3.10/dist-packages/requests/sessions.py", line 589, in

ValueError: Failed to load any schedule data.

In [12]:
# Pull data and store it in the data directory

sessionDateMap = {'conventional':{'FP1': 'Session1Date',
            'FP2': 'Session2Date',
            'FP3': 'Session3Date',
            'Qualifying': 'Session4Date',
            'Race': 'Session5Date',},
            'sprint':{'FP1':  'Session1Date',
            'Qualifying': 'Session2Date',
            'FP2': 'Session3Date',
            'Sprint': 'Session4Date',
            'Race': 'Session5Date'},
            'sprint_shootout':{'FP1': 'Session1Date',
            'Qualifying': 'Session2Date',
            'Sprint Shootout': 'Session3Date',
            'Sprint': 'Session4Date',
            'Race': 'Session5Date'}}

for year in [2019, 2020, 2021, 2022, 2023]:
    laps = []
    weathers = []
    results = []

    sch = fastf1.get_event_schedule(year).to_dict()
    sch = pd.DataFrame.from_dict(sch)

    EventName = [s for s in sch['EventName'] if 'Grand' in s]

    for event in EventName:

        event_type = sch.loc[sch['EventName']==event,'EventFormat'].values[0]

        for session_type in ['FP1','FP2','FP3','Sprint','Sprint Shootout','Qualifying','Race']:

            session = None

            try:
                session = fastf1.get_session(year, event, session_type) # call fastf1 to extract data
            except:
                print('Session: {} does not exist {}.'.format(session_type, event))
                session = None
            if session is not None:

                eventDateColName = sessionDateMap[event_type][session_type]

                try:
                    session.load()
                except:
                    print('Session: ' + session_type + ' does not provide usable data {}.'.format(event))
                    session = None

            # if the session exists, extract all data with auxiliary functions from the above cells
            if session is not None:
                try:
                    ## get lap data for a session
                    laps.append(get_lap_data(session, sch, event,
                                             session_type, eventDateColName))

                    ## get weather data for a session
                    weathers.append(get_weather_data(session, sch, event,
                                                     session_type, eventDateColName))

                    ## get results for a session
                    results.append(get_results(session, sch, event,
                                               session_type, eventDateColName))
                except DataNotLoadedError:
                    print('Trying to access data that isn\'t loaded yet')
                    continue


    ## save all extracted session data in a year
    file = open('{}{}_{}.pkl'.format(data_path, 'laps', year), 'wb')
    pickle.dump(laps,file)
    file = open('{}{}_{}.pkl'.format(data_path, 'weathers', year), 'wb')
    pickle.dump(weathers,file)
    file = open('{}{}_{}.pkl'.format(data_path, 'results', year), 'wb')
    pickle.dump(results,file)

core           INFO 	Loading data for Australian Grand Prix - Practice 1 [v3.3.1]
INFO:fastf1.fastf1.core:Loading data for Australian Grand Prix - Practice 1 [v3.3.1]
req            INFO 	No cached data found for session_info. Loading data...
INFO:fastf1.fastf1.req:No cached data found for session_info. Loading data...
_api           INFO 	Fetching session info data...
INFO:fastf1.api:Fetching session info data...
req            INFO 	Data has been written to cache!
INFO:fastf1.fastf1.req:Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
INFO:fastf1.fastf1.req:No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
INFO:fastf1.api:Fetching driver list...
req            INFO 	Data has been written to cache!
INFO:fastf1.fastf1.req:Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
INFO:fastf1.fastf1.req:No cached data found fo

Session: Sprint does not exist Australian Grand Prix.
Session: Sprint Shootout does not exist Australian Grand Prix.


req            INFO 	Data has been written to cache!
INFO:fastf1.fastf1.req:Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
INFO:fastf1.fastf1.req:No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
INFO:fastf1.api:Fetching driver list...
req            INFO 	Data has been written to cache!
INFO:fastf1.fastf1.req:Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
INFO:fastf1.fastf1.req:No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
INFO:fastf1.api:Fetching session status data...
req            INFO 	Data has been written to cache!
INFO:fastf1.fastf1.req:Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
INFO:fastf1.fastf1.req:No cached data found for track_status_data. Loading data...
_api  

Session: Sprint does not exist Bahrain Grand Prix.
Session: Sprint Shootout does not exist Bahrain Grand Prix.


req            INFO 	Data has been written to cache!
INFO:fastf1.fastf1.req:Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
INFO:fastf1.fastf1.req:No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
INFO:fastf1.api:Fetching driver list...
req            INFO 	Data has been written to cache!
INFO:fastf1.fastf1.req:Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
INFO:fastf1.fastf1.req:No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
INFO:fastf1.api:Fetching session status data...
req            INFO 	Data has been written to cache!
INFO:fastf1.fastf1.req:Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
INFO:fastf1.fastf1.req:No cached data found for track_status_data. Loading data...
_api  

...

Session: Qualifying does not exist Abu Dhabi Grand Prix.


logger      WARNING 	Failed to load schedule from Ergast API backend!
DEBUG:fastf1.fastf1.events:Traceback for failure in Ergast API Schedule
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/fastf1/logger.py", line 151, in __wrapped
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/fastf1/events.py", line 687, in _get_schedule_from_ergast
    season = fastf1.ergast.fetch_season(year)
  File "/usr/local/lib/python3.10/dist-packages/fastf1/ergast/legacy.py", line 29, in fetch_season
    Cache.requests_get(url, headers=_headers))
  File "/usr/local/lib/python3.10/dist-packages/fastf1/req.py", line 294, in requests_get
    return cls._requests_session_cached.get(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/requests_cache/session.py", line 126, in get
    return self.request('GET', url, params=params, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/requests_cache/session.py", line 182, in request


Session: Race does not exist Abu Dhabi Grand Prix.


logger      WARNING 	Failed to load schedule from FastF1 backend!
DEBUG:fastf1.fastf1.events:Traceback for failure in FastF1 schedule
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/fastf1/logger.py", line 151, in __wrapped
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/fastf1/events.py", line 556, in _get_schedule_ff1
    response = Cache.requests_get(
  File "/usr/local/lib/python3.10/dist-packages/fastf1/req.py", line 294, in requests_get
    return cls._requests_session_cached.get(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/requests_cache/session.py", line 126, in get
    return self.request('GET', url, params=params, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/requests_cache/session.py", line 182, in request
    return super().request(method, url, *args, headers=headers, **kwargs)  # type: ignore
  File "/usr/local/lib/python3.10/dist-packages/requests/sessions.py", line 589, in

ValueError: Failed to load any schedule data.

In [ ]:
def concat_data_and_save(data_source, data_path):
    '''
    e.g. data_source is weather
    1. finds all pkl files in the data_path folder
    2. concat all
    3. saves the in 'data_source'.pickle
    '''
    data = []
    files = os.listdir(data_path)
    file = [x for x in files if x.endswith('.pkl') if data_source in x]

    for fl in file:
        print(fl)
        file = open('{}{}'.format(data_path, fl), 'rb')
        data.extend(pickle.load(file))
    data = pd.concat(data, axis=0)

    file = open('{}{}_data.pickle'.format(data_path, data_source), 'wb')
    pickle.dump(data,file)
    return

In [ ]:
concat_data_and_save('weathers', data_path)
concat_data_and_save('laps', data_path)

In [ ]:
concat_data_and_save('results', data_path)

Done with data extraction. Process weather data.

In [ ]:
file = open('{}{}'.format(data_path, 'weathers_data.pickle'),'rb')
df_weather = pickle.load(file)

In [ ]:
df_weather['eventYear'] = pd.to_datetime(df_weather['EventDate'], utc=True).dt.year # get the year
df_weather['eventHour'] = df_weather['EventDate'].astype(str).str[11:13] # get the local hour
average_weather = df_weather.groupby(['EventName', 'RoundNumber','country', 'session', 'eventYear','eventHour'])['AirTemp','Humidity','Pressure','Rainfall','TrackTemp','WindDirection','WindSpeed'].mean().reset_index()

In [ ]:
df_weather.head()

In [ ]:
average_weather[average_weather['Rainfall']!= 0]

Process results data.

In [ ]:
file = open(data_path+'results_data.pickle','rb')
df_res = pickle.load(file)

In [ ]:
df_res['eventYear'] = pd.to_datetime(df_res['EventDate'], utc=True).dt.year
df_res['eventHour'] = df_res['EventDate'].astype(str).str[11:13] # get the local hour

## keep non-FP data
df_res = df_res[(df_res['session']=='Race') | (df_res['session']=='Qualifying') | (df_res['session']=='Sprint Shootout') | (df_res['session']=='Sprint Qualifying') | (df_res['session']== 'Sprint')]

df_res= df_res[['TeamName','FullName','EventName', 'RoundNumber','country', 'session', 'eventYear','eventHour', 'Position','ClassifiedPosition']].reset_index()

In [ ]:
df_res.head()

In [ ]:
#input variables:
#driver, team, track, race format, free practice fastest lap time, free practice position, free practice total number of laps,
#fp fastest lap weather conditions (avg over lap), fp avg weather conditions over sessions
#quali avg weather conditions, quali variance in weather conditions
#(sprint) race avg weather conditions, race variance in weather conditions
#last race result
#years on grid? testing results???

#output:
#qualifying position (binary % likelihood to be p1)
#race result (binary % likelihood to be p1)
#likelihood of dnf? (binary % likelihood to have dnfed)
#fastest lap? (binary % likelihood to have gotten fastest lap)

#translate into total points for each driver and constructor for the race (we'll ignore overtakes and fastest pit stops for now)

#input these + cost of each for fantasy team optimiser

#final data should include columns of input variables, quali result, sprint race result, race result